### Calculation of PER using machine learning

**What is PER?**  
The Price-to-Earnings ratio (P/E ratio) in finance is a key measure used to assess the valuation of a company in the stock market.  
The PER provides an indication of the number of years it would take to recoup the initial investment if the company were to distribute all its profits to shareholders as dividends.  
A high PER may indicate that the market values the company at a higher level relative to its current earnings, which may imply that investors anticipate future growth. Conversely, a low PER may indicate that the company is undervalued relative to its current earnings, which may represent an investment opportunity.

**How is PER calculated?**  
$$PER = Stock Price / Earnings Per Share$$

**Classic PER analysis:**  
To determine if a company has a high or low PER, it should be compared to the average PER of companies in the same sector.  
If our company has a PER lower (or higher) than the average, then our company is undervalued (or overvalued).

**PER analysis with machine learning:**  
In our study, to find comparable companies, we will consider:
- *The company's sector*
- *Market capitalization*
- *Total assets*
- *Revenue*

We will use a machine learning model to determine what should be the PER of our company according to the aforementioned criteria.  
As input, we will use the logarithm of market capitalization, total assets, and revenue (log-transformed for a normal distribution), as well as the company's sector (dummy variable takes 1 for the studied company's sector and 0 otherwise).  
And as Output, the logarithm of the PER (which we will exponentiate to obtain our predicted PER).

Once our model is trained on a sufficiently large sample, we can calculate what should be the PER of the company according to our machine learning model and compare it with the actual PER of the company.  
- If Actual PER < Predicted PER: action undervalued  
- If Actual PER > Predicted PER: action overvalued

Then we will look at the median of spreads by sector to determine which sector would be rather undervalued or overvalued:  
$spread = predicted PER - actual PER$  
- If spread < 0: sector undervalued  
- If spread > 0: sector overvalued

**Results:** the mean absolute error (MAE) of our machine learning model is still too high (a deviation of 0.89 around the log PER) and therefore needs to be improved.

In [8]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, regularizers
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from yahooquery import Ticker

In [2]:
# We load our dataset with information about companies
df_dataset = pd.read_csv("Dataset French Companies.csv")

list_tickers = df_dataset["Ticker"].tolist()
dic_data = {}
for ticker in list_tickers:
    print(ticker)
    action = Ticker(ticker)
    try:
        # We retrieve the market cap
        df_valuation_measures = action.valuation_measures
        mrkt_cap = df_valuation_measures.loc[df_valuation_measures['periodType'] == "TTM"]["MarketCap"].dropna()[-1]
        
        # We retrieve the PER
        per = df_valuation_measures.loc[df_valuation_measures["periodType"] == "TTM"]["PeRatio"].dropna()[-1]
        
        # We retrieve the Total Assets
        df_balance_sheet = action.balance_sheet()
        assets = df_balance_sheet.loc[df_balance_sheet["periodType"] == "12M"]["TotalAssets"].dropna()[-1]
        
        # We retrieve the Total Revenues
        df_income_statement = action.income_statement()
        revenue = df_income_statement.loc[df_income_statement["periodType"] == "12M"]["TotalRevenue"].dropna()[-1]
        
        if revenue <= 0 or assets <= 0 or mrkt_cap <= 0 or per <= 0:
            continue

        data_list = {"mkrt_cap": float(mrkt_cap), "assets": float(assets), "revenue": float(revenue), "sector": df_dataset.loc[df_dataset['Ticker'] == ticker, 'Sector'].iloc[0], "per": float(per)}
        name = df_dataset.loc[df_dataset['Ticker'] == ticker, 'Name'].iloc[0]
        dic_data[name] = data_list
    except:
        None


ALAGP.PA
ALBOO.PA
CGG.PA
ALCWE.PA
ALDOL.PA
ALESA.PA
ALENE.PA
ALENO.PA
ALESE.PA
ALEO2.PA
ES.PA
ALGBE.PA
GTT.PA
ALHAF.PA
ALHRS.PA
LHYFE.PA
MAU.PA
MCPHY.PA
NHOA.PA
MLSEQ.PA
SLB.PA
TE.PA
EC.PA
TTE.PA
VK.PA
ALVER.PA
WAGA.PA
ALAFY.PA
AI.PA
ALMIB.PA
AKE.PA
ALAMG.PA
ALBKK.PA
ALCRB.PA
ALCOG.PA
ALCOI.PA
ALDUB.PA
ERA.PA
ALEXA.PA
EXPL.PA
FALG.PA
ALFLO.PA
ALGLD.PA
ALKEM.PA
NK.PA
JCQ.PA
METEX.PA
ALMOU.PA
ORAP.PA
RBT.PA
CBE.PA
ALRGR.PA
MLZAM.PA
AB.PA
ABNX.PA
ABVX.PA
ALACT.PA
ADOC.PA
ALDVI.PA
AELIS.PA
AFME.PA
AMPLI.PA
BLC.PA
BIM.PA
ALBPS.PA
ALBIO.PA
ALBLU.PA
BOI.PA
MLBON.PA
ALCAR.PA
ALCLS.PA
CLARI.PA
CLARI.PA
ALCJ.PA
DBV.PA
ALDMS.PA
EL.PA
EAPI.PA
ALERS.PA
ERF.PA
ALEMG.PA
GNRO.PA
GNFT.PA
ALGEN.PA
SIGHT.PA
GBT.PA
ALHYG.PA
ALICR.PA
ALIKO.PA
ALIMP.PA
IPH.PA
ALINT.PA
ALINS.PA
IVA.PA
IPN.PA
LNA.PA
LBIRD.PA
MAAT.PA
ALMKT.PA
ALMDP.PA
MEDCL.PA
NANO.PA
ALNEV.PA
ALNFL.PA
ALCOX.PA
ALNOV.PA
ALOPM.PA
ORP.PA
OSE.PA
PHXM.PA
ALPAT.PA
POXEL.PA
ALPRE.PA
ALQGC.PA
GDS.PA
ALSAF.PA
SAN.PA
DIM.PA
ALSEN.PA
ALSMA.PA
ALSGD.PA
A

In [3]:
# We create a DataFrame to format our data
df = pd.DataFrame(dic_data).T
df["log_mkrt_cap"] = np.log(list(df["mkrt_cap"]))
df["log_assets"] = np.log(list(df["assets"]))
df["log_revenue"] = np.log(list(df["revenue"]))
df["log_per"] = np.log(list(df["per"]))

# We remove extreme values (60 is arbitrary)
df = df.query("per < 60").copy()

# We create a list with all the sectors
sector_list = list(set(df["sector"]))

# We create a dummy variable for the sector
for sector in sector_list:
    df[sector] = np.where(df["sector"] == sector, 1, 0)
df

,mkrt_cap,assets,revenue,sector,per,log_mkrt_cap,log_assets,log_revenue,log_per,Services aux Collectivités,Services aux consommateurs,Télécommunications,Sociétés financières,Pétrole et Gaz,Santé,Matériaux de base,Biens de consommation,Industries,Technologies
DOLFINES,2367270.0,9163945.0,4163083.0,Pétrole et Gaz,0.005652,14.677248,16.030787,15.241766,-5.175746,0,0,0,0,1,0,0,0,0,0
EO2,11766287.0,47030000.0,40050000.0,Pétrole et Gaz,2.684492,16.280749,17.666296,17.505639,0.987492,0,0,0,0,1,0,0,0,0,0
ESSO,1068215431.0,4493000000.0,24936000000.0,Pétrole et Gaz,0.433294,20.789255,22.225786,23.939578,-0.836339,0,0,0,0,1,0,0,0,0,0
GTT (GAZTRANSPORT ET TEC.),5284289296.0,484445000.0,307294000.0,Pétrole et Gaz,26.335175,22.388004,19.998514,19.543316,3.270905,0,0,0,0,1,0,0,0,0,0
MAUREL & PROM,1050415766.0,1677664000.0,676480000.0,Pétrole et Gaz,9.47769,20.772452,21.240668,20.332413,2.248941,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HF COMPANY,16420887.0,30833000.0,6432000.0,Télécommunications,6.571429,16.614065,17.244096,15.676796,1.882731,0,0,1,0,0,0,0,0,0,0
MAROC TELECOM,7972642235.0,65543000000.0,37924000000.0,Télécommunications,17.223375,22.799282,24.905972,24.358850,2.846267,0,0,1,0,0,0,0,0,0,0
ORANGE,28187366742.0,109650000000.0,43471000000.0,Télécommunications,12.475294,24.062140,25.420559,24.495360,2.523750,0,0,1,0,0,0,0,0,0,0
OSMOZIS,30638221.0,34025302.0,12696217.0,Télécommunications,38.965517,17.237759,17.342615,16.356815,3.662677,0,0,1,0,0,0,0,0,0,0


In [4]:
# We create our machine learning model
X_list = sector_list + ["log_mkrt_cap", "log_assets", "log_revenue"]
X = df[X_list]
y = df["log_per"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

stock_model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1)
])

stock_model.compile(loss=tf.keras.losses.mae,
                   optimizer=tf.keras.optimizers.Adamax(),
                   metrics=["mae"])

stock_model.fit(X_train, y_train, epochs=50)

stock_model.evaluate(X_test, y_test)[1]


Epoch 1/50


10/10 [==============================] - 2s 6ms/step - loss: 8.9507 - mae: 8.3115
Epoch 2/50
10/10 [==============================] - 0s 6ms/step - loss: 2.9201 - mae: 2.3300
Epoch 3/50
10/10 [==============================] - 0s 5ms/step - loss: 1.9291 - mae: 1.3824
Epoch 4/50
10/10 [==============================] - 0s 6ms/step - loss: 1.2908 - mae: 0.7834
Epoch 5/50
10/10 [==============================] - 0s 6ms/step - loss: 1.2261 - mae: 0.7534
Epoch 6/50
10/10 [==============================] - 0s 5ms/step - loss: 1.1366 - mae: 0.6945
Epoch 7/50
10/10 [==============================] - 0s 5ms/step - loss: 1.0934 - mae: 0.6787
Epoch 8/50
10/10 [==============================] - 0s 5ms/step - loss: 1.0616 - mae: 0.6713
Epoch 9/50
10/10 [==============================] - 0s 5ms/step - loss: 1.0325 - mae: 0.6640
Epoch 10/50
10/10 [==============================] - 0s 5ms/step - loss: 1.0072 - mae: 0.6581
Epoch 11/50
10/10 [==============================] - 0s 5ms/step -

0.9026236534118652

In [5]:
# We apply our machine learning model to each company in the dataset to calculate their predicted PER
dic_df_verdict = {}
for ticker in df.T.columns:
    predicted_per = np.exp(stock_model.predict(df[X_list].loc[ticker].to_frame().T))
    actual_per = df.loc[ticker, "per"]
    if actual_per < predicted_per: 
        status = "undervalued"
    else: 
        status = "overvalued"
    dic_df_verdict[ticker] = {"predicted_per": predicted_per[0][0], "actual_per": actual_per, "status": status}

1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 56ms/step


1/1 [==============================] - 0s 43ms/step


In [6]:
# We calculate the spread between the predicted PER and the actual PER of the company
df_per = pd.DataFrame(dic_df_verdict).T
df_per["spread"] = df_per["actual_per"] - df_per["predicted_per"]
df_per.sort_values(by="spread")

,predicted_per,actual_per,status,spread
STMICROELECTRONICS,36.648674,10.668499,undervalued,-25.980175
CATERPILLAR,34.55669,17.088349,undervalued,-17.468341
ALTEN,27.359627,11.210654,undervalued,-16.148973
STELLANTIS,19.740011,4.223906,undervalued,-15.516105
X-FAB SILICON,21.116375,6.556623,undervalued,-14.559752
...,...,...,...,...
PROACTIS,10.116642,53.713218,overvalued,43.596576
ARTOIS N,12.539551,56.608243,overvalued,44.068692
ARTMARKET.COM,12.026017,56.896552,overvalued,44.870535
HYBRIGENICS,6.236348,54.577465,overvalued,48.341117


In [7]:
# Here we will look at the median of spreads (between actual PER and predicted PER) for each sector
df_per_sector = pd.concat([df_per, pd.DataFrame(dic_data).T["sector"].to_frame()], axis=1).dropna()
dic_median = {}
for sector in sector_list:
    dic_median[sector] = df_per_sector.loc[df_per_sector["sector"] == sector]["spread"].median()
dic_median_final = {"Median spread per sector": dic_median}
pd.DataFrame(dic_median_final).sort_values(by="Median spread per sector")

,Median spread per sector
Pétrole et Gaz,-4.455404
Télécommunications,-1.091709
Services aux consommateurs,-0.397840
Sociétés financières,-0.377348
Industries,-0.273892
Biens de consommation,0.233506
Technologies,0.696861
Matériaux de base,0.735459
Services aux Collectivités,1.431267
Santé,2.276705
